In [2]:
import os

import networkx as nx
from pymongo import MongoClient

In [13]:
client = MongoClient()
# client.drop_database('recipes')
db = client.recipes

In [14]:
ingredients_graph = nx.read_gexf('data/spanish_ingredients_lexicon_4.gexf')

In [15]:
techniques_graph = nx.read_gexf('data/spanish_techniques_lexicon_4.gexf')

In [29]:
path = 'data/recipes/elbulli/'
for folder in os.listdir(path):
    for filename in os.listdir(path + folder):
        c+=1
        with open(path + folder + '/' + filename) as f:
            info = {}
            for line in f:
                line = line.strip()
                if line.startswith('num'):
                    d+=1
                    info['ide'] = line.split('=')[1]
                elif line.startswith('&any'):
                    e+=1
                    info['year'] = line.split('=')[1]
            print(info)
            break
        break

{'ide': '40', 'year': '1988'}
{'ide': '1138', 'year': '2005'}
{'ide': '482', 'year': '1998'}
